In [ ]:
class CalCalcAPI(Resource):
    def post(self):
        try:
            data = request.get_json()

            # Ensure that all required fields are present in the request JSON
            required_fields = ['age', 'weight', 'height', 'activity', 'gender', 'username']
            for field in required_fields:
                if field not in data:
                    return {'error': f"Missing field: {field}"}, 400

            # Calculate calorie maintenance (you can reuse your client-side logic here)
            calorie_maintenance = calculateCalories(data)

            # Calculate the meal plan based on calorie maintenance
            mealPlan = ''
            mealPlanNumber = 0
            if calorie_maintenance < 1000:
                mealPlan = 'Meal Plan 1'
            else:
                mealPlanNumber = (calorie_maintenance - 1000) // 250 + 1
                mealPlan = f'Meal Plan {mealPlanNumber}'


In [ ]:
import threading

# import "packages" from flask
from flask import render_template  # import render_template from "public" flask libraries
from flask_sqlalchemy import SQLAlchemy

# import "packages" from "this" project
from __init__ import app,db  # Definitions initialization
from model.jokes import initJokes
from model.users import initUsers
from model.players import initPlayers
from model.calcalcs import initCalories

# setup APIs
from api.covid import covid_api # Blueprint import api definition
from api.joke import joke_api # Blueprint import api definition
from api.user import user_api # Blueprint import api definition
from api.player import player_api
from api.calcalc import calcalc_api

# setup App pages
from projects.projects import app_projects # Blueprint directory import projects definition


# Initialize the SQLAlchemy object to work with the Flask app instance
db.init_app(app)

# register URIs
app.register_blueprint(calcalc_api)
app.register_blueprint(joke_api) # register api routes
app.register_blueprint(covid_api) # register api routes
app.register_blueprint(user_api) # register api routes
app.register_blueprint(player_api)
app.register_blueprint(app_projects) # register app pages

@app.errorhandler(404)  # catch for URL not found
def page_not_found(e):
    # note that we set the 404 status explicitly
    return render_template('404.html'), 404

@app.route('/')  # connects default URL to index() function
def index():
    return render_template("index.html")

@app.route('/table/')  # connects /stub/ URL to stub() function
def table():
    return render_template("table.html")



@app.before_first_request
def activate_job():  # activate these items 
    initJokes()
    initUsers()
    initPlayers()

# this runs the application on the development server
if __name__ == "__main__":
    # change name for testing
    from flask_cors import CORS
    cors = CORS(app)
    app.run(debug=True, host="0.0.0.0", port="8169")
